In [1]:
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;
using Microsoft.DotNet.Interactive.Formatting;
using Microsoft.DotNet.Interactive;
//using XPlot.Plotly.Interactive;
using System.Collections.Generic;
using System.IO;
using System;
using System.Text.RegularExpressions;

In [2]:
var fileExists = File.Exists("input.txt");
if (fileExists)
{
    display("This file exists.  Nothing has blown up.....yet.");
}

This file exists.  Nothing has blown up.....yet.

In [3]:
string[] lines = System.IO.File.ReadAllLines("input.txt");
lines.Length
//lines[0].Length


140

In [29]:
public class CharacterGrid
{
    public char[,] grid; // Internal 2D array for storing characters
    public int rows; // Number of rows in the grid
    public int columns; // Number of columns in the grid

    public CharacterGrid(int rows, int columns)
    {
        this.rows = rows;
        this.columns = columns;
        this.grid = new char[rows, columns];
    }

    // Get the character at a specific row and column
    public char GetCharacter(int row, int column)
    {
        if (!IsValidIndex(row, column))
        {
            return '.'; // return a useless '.' character if the index isn't valid
        }

        return grid[row, column];
    }

    // Set the character at a specific row and column
    public void SetCharacter(int row, int column, char character)
    {
        if (!IsValidIndex(row, column))
        {
            throw new IndexOutOfRangeException("Invalid row or column index");
        }

        grid[row, column] = character;
    }

    public void SetRow(string line, int rowNum)
    {
        int i = 0;
        foreach(char c in line)
        {
            SetCharacter(rowNum, i, c);
            i++;
        }
    }

    public List<PartNumber> GetUnverifiedParts(string line, int rowNum)
    {
        List<PartNumber> unverifiedParts = new List<PartNumber>();

        Regex regex = new Regex(@"(\d+)");

        foreach (Match match in regex.Matches(line))
        {
            int number = int.Parse(match.Groups[1].Value);
            int startPos = match.Index;
            int endPos = startPos + match.Length;
            PartNumber tempPart = new PartNumber(number, startPos, endPos, rowNum);
            unverifiedParts.Add(tempPart);
            //Console.WriteLine($"Line: {rowNum} Number: {number}, Start: {startPos}, End: {endPos}");
        }
        return unverifiedParts;
    }

    public List<PartNumber> GetVerifiedParts(List<PartNumber> unverifiedParts)
    {
        List<PartNumber> VerifiedParts = new List<PartNumber>();

        foreach (PartNumber pn in unverifiedParts)
        {
            bool isSpecial = false;
            foreach ((int row, int column) bc in pn.borderCharacters)
            {
                //Console.WriteLine("Looking for special character for value " + pn.value + " at position: " + bc.row + "/" + bc.column);
                bool actuallyIsSpecial = IsSpecialCharacter(bc.row, bc.column);
                if (actuallyIsSpecial)
                {
                    isSpecial = true;
                    break;
                }
            }
            if(isSpecial)
            {
                pn.isValid = true;
                VerifiedParts.Add(pn);
            }
            else
            {
                Console.WriteLine("Part value " + pn.value + " is not valid");
            }

        }
        return VerifiedParts;
    }

    public bool IsSpecialCharacter(int row, int column)
    {
        // looking for *, =, -, +, &, #, %, /, @, $
        char c = GetCharacter(row, column);
        bool special =  false;
        switch(c)
        {
            case '*':
            case '=':
            case '-':
            case '+':
            case '&':
            case '#':
            case '%':
            case '/':
            case '@':
            case '$':
                //Console.Out.WriteLine("Character " + c + " is special");
                special = true;
                break;
            default:
                special = false;
                break;
        }
        return special;

    }

    // Check if a row and column index are within the grid bounds
    public bool IsValidIndex(int row, int column)
    {
        return row >= 0 && row < rows && column >= 0 && column < columns;
    }

    public bool HasValidIndicator()
    {
        return false;
    }


    
}

public class PartNumber
{
    public int value;
    public int startColumn;
    public int endColumn;
    public int row;
    public bool isValid;
    public List<(int row, int column)> borderCharacters;

    public PartNumber(int value, int startColumn, int endColumn, int row)
    {
        this.value = value;
        this.startColumn = startColumn;
        this.endColumn = endColumn;
        this.row = row;
        borderCharacters = GetBorderCharacters();
    }

    public List<(int row, int column)> GetBorderCharacters()
    {
        List<(int row, int column)> borderChars = new List<(int row, int column)>();
        int leftColumn = this.startColumn - 1;
        int rightColumn = this.endColumn;
        int topRow = this.row - 1;
        int bottomRow = this.row + 1;
        //Console.WriteLine($"TopRow: {topRow} BottomRow: {bottomRow}, LeftColumn: {leftColumn}, RightColumn: {rightColumn}");
        // Example - TopRow: 0 BottomRow: 2, LeftColumn: 2, RightColumn: 6
        // Add top and bottom rows.
        for (int i = leftColumn; i<=rightColumn; i++)
        {
            (int row, int column) temp = (topRow, i);
            (int row, int column) temp2 = (bottomRow, i);
            borderChars.Add(temp);
            borderChars.Add(temp2);
        }
        // Add middle row. Only add leftmost and rightmost columns.
        borderChars.Add((this.row, leftColumn));
        borderChars.Add((this.row, rightColumn));




        return borderChars;
    }

}



In [5]:
// Let's get a list of all characters, to specify the "special" ones.
List<char> allChars = lines.SelectMany(s => s.ToCharArray()).Distinct().ToList();

Console.WriteLine(string.Join(", ", allChars));

4, 8, ., 5, 0, 1, 3, 6, 2, 7, 9, *, =, -, +, &, #, %, /, @, $


In [30]:
CharacterGrid cg = new CharacterGrid(140, 140);
int i = 0;
List<PartNumber> parts = new List<PartNumber>();
foreach(string line in lines)
{
    cg.SetRow(line, i);
    List<PartNumber> tempParts = cg.GetUnverifiedParts(line, i);
    parts.AddRange(tempParts);
    i++;
}


//char x = cg.GetCharacter(137,136);

//bool isSpecial = cg.IsSpecialCharacter(137,136);

//PartNumber p = new PartNumber(48, 0, 1, 0);
//PartNumber p2 = new PartNumber(491,3, 5, 1);

//List<(int row, int column)> d = p2.GetBorderCharacters();

List<PartNumber> validParts = cg.GetVerifiedParts(parts);

int sum = 0;
foreach(PartNumber p in validParts)
{
    sum += p.value;
}

sum


Part value 48 is not valid
Part value 33 is not valid
Part value 622 is not valid
Part value 426 is not valid
Part value 843 is not valid
Part value 306 is not valid
Part value 597 is not valid
Part value 861 is not valid
Part value 864 is not valid
Part value 475 is not valid
Part value 729 is not valid
Part value 662 is not valid
Part value 414 is not valid
Part value 933 is not valid
Part value 726 is not valid
Part value 802 is not valid
Part value 474 is not valid
Part value 224 is not valid
Part value 615 is not valid
Part value 572 is not valid
Part value 995 is not valid
Part value 499 is not valid
Part value 574 is not valid
Part value 778 is not valid
Part value 236 is not valid
Part value 593 is not valid
Part value 36 is not valid
Part value 226 is not valid
Part value 790 is not valid
Part value 248 is not valid
Part value 173 is not valid
Part value 95 is not valid
Part value 877 is not valid
Part value 497 is not valid
Part value 427 is not valid
Part value 761 is not va

521601

My initial answer of 491927 is wrong. However, I'm not sure what will make it right. It says that my answer (491927) is too low. I assume that my search for special characters in the border characters, so I'll need to debug that.
I was right about the border characters. The calculation was right but the end position was wrong, as evidenced by the sample data.
My new answer of 519543 is also too low, so I must be missing some other special character detection.

In [22]:
string[] sample = System.IO.File.ReadAllLines("sample.txt");

CharacterGrid cgs = new CharacterGrid(10, 10);
int i = 0;
List<PartNumber> parts = new List<PartNumber>();
foreach(string line in sample)
{
    cgs.SetRow(line, i);
    List<PartNumber> tempParts = cgs.GetUnverifiedParts(line, i);
    parts.AddRange(tempParts);
    i++;
}


//char x = cgs.GetCharacter(137,136);

//bool isSpecial = cgs.IsSpecialCharacter(137,136);

//PartNumber p = new PartNumber(48, 0, 1, 0);
//PartNumber p2 = new PartNumber(491,3, 5, 1);

//List<(int row, int column)> d = p2.GetBorderCharacters();

List<PartNumber> validParts = cgs.GetVerifiedParts(parts);

//validParts

int sum = 0;
foreach(PartNumber p in validParts)
{
    sum += p.value;
}
sum

Part value 114 is not valid
Part value 58 is not valid


4361